In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#importing libraries
import cudf, cupy, cuml

In [3]:
#importing the data
train = cudf.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = cudf.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [4]:
#Counting number of categorical and numerical features
numerical_features = train.dtypes[train.dtypes != "object"].index
print("Number of Numerical features: ", len(numerical_features))
categorical_features = train.dtypes[train.dtypes == "object"].index
print("Number of Categorical features: ", len(categorical_features))

In [5]:
#Listing all the categorical Features
for catg in list(categorical_features) :
    print(train[catg].value_counts())
    print('-'*50)

In [6]:
#dropping the categorical features
train.drop(categorical_features, axis = 1, inplace = True)
test.drop(categorical_features, axis = 1, inplace = True)

In [7]:
#dropping null value columns
train.drop(['LotFrontage', 'MasVnrArea','GarageYrBlt'], axis = 1, inplace = True)
test.drop(['LotFrontage', 'MasVnrArea','GarageYrBlt'], axis = 1, inplace = True)

In [8]:
#Splitting the independent and dependent variable
X = train.drop(['SalePrice'],axis=1)
Y = train['SalePrice']

In [9]:
#Splitting the dataset
from cuml.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

In [10]:
#All 5 linear Regression Algorithms
algorithm = ['svd', 'eig', 'svd-qr', 'svd-jacobi', 'qr']

In [11]:
from cuml.linear_model import LinearRegression
sample_sub = cudf.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')
for i in algorithm:
    print("Algorithm:")
    print(i)
    linear_regressor = LinearRegression(fit_intercept = True, normalize = False, algorithm = i)
    regressor = linear_regressor.fit(X_train.astype('float32'), y_train.astype('float32'))
    predicted_value = linear_regressor.predict(X_test)
    print("MSE:")
    print(cuml.metrics.regression.mean_squared_error(y_test.astype('int64'),predicted_value.astype('int64')))
    print("R2 Score:")
    print(cuml.metrics.regression.r2_score(y_test.astype('float32'), predicted_value.astype('float32')))
    print("MAE:")
    print(cuml.metrics.regression.mean_absolute_error(y_test.astype('int64'), predicted_value.astype('int64')))
    print('\n')
    sample_sub['SalePrice'] = predicted_value
    sample_sub.to_csv('submission_' + i + '.csv' , index = False)